In [1]:
ModelsDir <- '/home/kate/Research/Property/Models/'
DataDir <- '/home/kate/Research/Property/Data/'
ModelName <- "wc_poissonglm_f_ecy"
UseSavedIfExists <- FALSE

In [2]:
source('/home/kate/code/Utils/MyFunctions.R')

In [3]:
training_dataset <- read.csv(paste(DataDir,"property_wcf_training.csv", sep = ""), header=TRUE)
testing_dataset <- read.csv(paste(DataDir,"property_wcf_testing.csv", sep = ""), header=TRUE)

In [4]:
formula <- cova_ic_nc_water ~ log_ecy + log_sqft + usagetype + log_property_age + log_water_risk_3_blk + cova_deductible + customer_cnt_active_policies 

In [5]:
kfold <- 5

In [6]:
models_attr <- data.frame()
models_coef <- data.frame()
training_dataset$poissonglm_f_ecy_cv <- 0
testing_dataset$poissonglm_f_ecy_cv <- 0

for (fold in 1:kfold-1){
  training_dataset[paste("poissonglm_f_ecy_",fold,sep="")] <- 0 
  testing_dataset[paste("poissonglm_f_ecy_",fold,sep="")] <- 0  
    
    }
# 
#CV-Folds
AIC_l <- list()
BIC_l <- list()
logLik_l <- list()
NWG_Test <- list()
NWG_Train <- list()
coef_l <- list()
folds_l <- list() 
i <- 1
for (fold in 1:kfold-1){
    print(paste("Fold ",fold," of ",kfold," currently processing...",sep=""))
    training_dataset_fold <- training_dataset[training_dataset[paste("fold",fold,sep="_")] > 0,]
    validation_dataset <- training_dataset[training_dataset[paste("fold",fold,sep="_")] == 0,]
    #Modeling
    ModelFile <- paste(ModelsDir,ModelName,"_",fold,".rds",sep="")
    if(file.exists(ModelFile) && UseSavedIfExists){
        poissonglm <- readRDS(ModelFile)
    } else {
        poissonglm <- glm(formula, family = "poisson", data = training_dataset_fold)
        saveRDS(poissonglm, ModelFile)             
    }#if modeling 
    #Training Prediction
    pred_train <- predict(poissonglm,training_dataset, type="response")
    train_fold_NWG <- NormalizedWeightedGini(training_dataset$cova_ic_nc_water,pred_train,training_dataset$ecy)    
    training_dataset[paste("poissonglm_f_ecy_",fold,sep="")] <- pred_train
    training_dataset$poissonglm_f_ecy_cv <- training_dataset$poissonglm_f_ecy_cv + pred_train/kfold
    #Testing Prediction
    pred_test <- predict(poissonglm,testing_dataset, type="response")
    test_fold_NWG <- NormalizedWeightedGini(testing_dataset$cova_ic_nc_water,pred_test,testing_dataset$ecy)    
    testing_dataset$poissonglm_f_ecy_cv <- testing_dataset$poissonglm_f_ecy_cv + pred_test/kfold 
    testing_dataset[paste("poissonglm_f_ecy_",fold,sep="")] <- pred_test

    #Model attributes
    folds_l[[i]] <- i
    AIC_l[[i]] <- AIC(poissonglm)
    BIC_l[[i]] <- BIC(poissonglm)  
    logLik_l[[i]] <- logLik(poissonglm) 
    NWG_Train[[i]] <- train_fold_NWG    
    NWG_Test[[i]] <- test_fold_NWG    
    #Model coeficients
    model_coef_fold <- data.frame(
        fold = fold+1,
        pred_name = summary(poissonglm)$coefficients[,0],
        estimate =  summary(poissonglm)$coefficients[,1],
        StdError =  summary(poissonglm)$coefficients[,2],  
        zValue =   summary(poissonglm)$coefficients[,3],     
        valuePr =   summary(poissonglm)$coefficients[,4]    
    )  
    #pred_name is an index, row name, convert it into column
    model_coef_fold <- cbind(name = rownames(model_coef_fold), model_coef_fold)
    rownames(model_coef_fold) <- 1:nrow(model_coef_fold) 
    #main table
    models_coef <- rbind(models_coef,model_coef_fold)         
    i <- i + 1
    }#folds loop   
    #Model attributes data frame
    models_fold_attr <- data.frame( 
        fold = unlist(folds_l),
        AIC = unlist(AIC_l),
        BIC = unlist(BIC_l),   
        logLik = unlist(logLik_l),
        TrainNormalizedWeightedGini = unlist(NWG_Train),     
        TestNormalizedWeightedGini = unlist(NWG_Test)
    )
    models_attr <- rbind(models_attr,models_fold_attr)                 

[1] "Fold 0 of 5 currently processing..."
[1] "Fold 1 of 5 currently processing..."
[1] "Fold 2 of 5 currently processing..."
[1] "Fold 3 of 5 currently processing..."
[1] "Fold 4 of 5 currently processing..."


In [7]:
head(models_attr)

,fold,AIC,BIC,logLik,TrainNormalizedWeightedGini,TestNormalizedWeightedGini
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,92317.25,92474.19,-46145.63,0.3786988,0.4083921
2,2,92203.78,92360.72,-46088.89,0.3786780,0.4090509
3,3,92298.01,92454.94,-46136.00,0.3787557,0.4090573
4,4,92268.00,92424.93,-46121.00,0.3787747,0.4093065
5,5,92429.78,92586.72,-46201.89,0.3782675,0.4086750


In [8]:
write.table(models_attr,paste(ModelsDir,ModelName,"_attr.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)

In [9]:
head(models_coef)

,name,fold,estimate,StdError,zValue,valuePr
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,(Intercept),1,-15.5175216,1.03848653,-14.942439,1.744708e-50
2,log_ecy,1,0.9676736,0.02172045,44.551268,0.000000e+00
3,log_sqft,1,0.6985639,0.03119335,22.394644,4.438449e-111
4,usagetypePRIMARY,1,3.4177480,1.00017639,3.417145,6.328150e-04
5,usagetypeRENTAL,1,3.1141342,1.00033829,3.113081,1.851451e-03
6,usagetypeSEASONAL,1,3.0702063,1.01845970,3.014558,2.573536e-03


In [10]:
write.table(models_coef,paste(ModelsDir,ModelName,"_coef.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)

Normalized Weighted gini Training

In [11]:
NormalizedWeightedGini(training_dataset$cova_ic_nc_water,training_dataset$poissonglm_f_ecy_cv,training_dataset$ecy)

[1] 0.3787112

Normalized Weighted gini Testing

In [12]:
NormalizedWeightedGini(testing_dataset$cova_ic_nc_water,testing_dataset$poissonglm_f_ecy_cv,testing_dataset$ecy)

[1] 0.4089942

In [ ]:
write.table(training_dataset,paste(DataDir,"property_wcf_training.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)
write.table(testing_dataset,paste(DataDir,"property_wcf_testing.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)
write.table(prediction_dataset,paste(DataDir,"property_water_claims_non_cat_fs.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)